# SW-5b-Python-LinkedData

**Sidetrack Python** : [<< SW-5-CSharp-LinkedData](SW-5-CSharp-LinkedData.ipynb) | [Index](README.md) | [SW-6-CSharp-RDFS >>](SW-6-CSharp-RDFS.ipynb)

## Donnees Liees du Web en Python avec SPARQLWrapper

Ce notebook est un **sidetrack optionnel** qui presente l'equivalent Python des concepts de donnees liees du notebook SW-5 (dotNetRDF). Vous y decouvrirez **SPARQLWrapper** pour interroger des endpoints publics comme DBpedia et Wikidata.

### Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Interroger des endpoints SPARQL distants avec SPARQLWrapper
2. Explorer DBpedia (donnees structurees de Wikipedia)
3. Interroger Wikidata (base de connaissances collaborative)
4. Faire la correspondance entre dotNetRDF et SPARQLWrapper

### Prerequis
- SW-5-CSharp-LinkedData recommande (pour la comprehension conceptuelle)
- Python 3.10+
- Connexion internet (pour acceder aux endpoints publics)

### Duree estimee : 25 minutes

> **Attention** : Les endpoints publics peuvent etre temporairement indisponibles. Toutes les requetes sont enveloppees dans des `try/except` pour gerer ces cas gracieusement.

---

## 1. Installation et Imports

**SPARQLWrapper** est la bibliotheque de reference pour interroger des endpoints SPARQL distants depuis Python.

In [ ]:
%pip install -q SPARQLWrapper

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON, XML
import json

print("SPARQLWrapper importe.")
print(f"Version : {SPARQLWrapper.__version__}")

---

## 2. Interroger DBpedia

DBpedia extrait des donnees structurees de Wikipedia et les expose sous forme de triples RDF. L'endpoint SPARQL est accessible a `http://dbpedia.org/sparql`.

In [ ]:
# Query DBpedia for French cities
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)

sparql.setQuery("""
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?city ?name ?population
WHERE {
    ?city a dbo:City .
    ?city dbo:country dbr:France .
    ?city rdfs:label ?name .
    ?city dbo:populationTotal ?population .
    FILTER (lang(?name) = "fr")
}
ORDER BY DESC(?population)
LIMIT 5
""")

try:
    results = sparql.query().convert()
    bindings = results["results"]["bindings"]

    print(f"=== Top 5 villes francaises (DBpedia) ===")
    print(f"{'Ville':<25s} {'Population':>12s}")
    print("-" * 37)
    for r in bindings:
        name = r["name"]["value"]
        pop = int(r["population"]["value"])
        print(f"{name:<25s} {pop:>12,d}")

except Exception as e:
    print(f"Erreur lors de la requete DBpedia : {type(e).__name__}: {e}")
    print("L'endpoint DBpedia est peut-etre temporairement indisponible.")
    print("Cela n'empeche pas de continuer le notebook.")

### Interpretation : Requete DBpedia

**Structure de la reponse** : SPARQLWrapper retourne un dictionnaire JSON contenant :
- `results.bindings` : liste de dictionnaires, un par ligne de resultat
- Chaque variable (`?city`, `?name`, `?population`) est une cle avec `value` et `type`

**Points cles** :
1. `FILTER (lang(?name) = "fr")` limite les labels a la langue francaise
2. `LIMIT 5` restreint le nombre de resultats (important pour les endpoints publics)
3. Le `try/except` est essentiel : les services publics peuvent avoir des timeouts

| Operation | SPARQLWrapper | dotNetRDF |
|-----------|---------------|----------|
| Creer endpoint | `SPARQLWrapper("http://...")` | `new SparqlRemoteEndpoint(uri)` |
| Format retour | `setReturnFormat(JSON)` | Constructeur parametre |
| Definir requete | `setQuery("...")` | `Query = "..."` |
| Executer | `query().convert()` | `QueryWithResultSet()` |

---

## 3. Interroger Wikidata

Wikidata est la base de connaissances structuree du projet Wikimedia. Son endpoint SPARQL est a `https://query.wikidata.org/sparql`.

**Difference importante** : Wikidata utilise des identifiants numeriques (Q-items, P-properties) plutot que des URI lisibles.

In [ ]:
# Query Wikidata for programming languages created after 2010
sparql_wd = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql_wd.setReturnFormat(JSON)

# Add a User-Agent header (required by Wikidata policy)
sparql_wd.agent = "CoursIA-SemanticWeb-Notebook/1.0 (educational)"

sparql_wd.setQuery("""
SELECT ?lang ?langLabel ?inception
WHERE {
    ?lang wdt:P31 wd:Q9143 .        # instance of: programming language
    ?lang wdt:P571 ?inception .      # inception date
    FILTER (YEAR(?inception) >= 2010)
    SERVICE wikibase:label { bd:serviceParam wikibase:language "fr,en" . }
}
ORDER BY DESC(?inception)
LIMIT 10
""")

try:
    results_wd = sparql_wd.query().convert()
    bindings_wd = results_wd["results"]["bindings"]

    print(f"=== Langages de programmation recents (Wikidata) ===")
    print(f"{'Langage':<25s} {'Annee de creation':>20s}")
    print("-" * 45)
    for r in bindings_wd:
        name = r["langLabel"]["value"]
        year = r["inception"]["value"][:4]  # extract year from date
        print(f"{name:<25s} {year:>20s}")

except Exception as e:
    print(f"Erreur lors de la requete Wikidata : {type(e).__name__}: {e}")
    print("L'endpoint Wikidata est peut-etre temporairement indisponible.")
    print("Cela n'empeche pas de continuer le notebook.")

### Interpretation : Requete Wikidata

**Differences DBpedia vs Wikidata** :

| Aspect | DBpedia | Wikidata |
|--------|---------|----------|
| **Identifiants** | URIs lisibles (`dbr:Paris`) | Q-items (`wd:Q90` = Paris) |
| **Proprietes** | Ontologie (`dbo:populationTotal`) | P-properties (`wdt:P1082`) |
| **Labels** | `rdfs:label` + `FILTER(lang())` | `SERVICE wikibase:label` |
| **User-Agent** | Optionnel | **Obligatoire** (politique Wikimedia) |

**Identifiants Wikidata courants** :
- `wd:Q5` = human
- `wd:Q9143` = programming language
- `wd:Q515` = city
- `wd:Q142` = France
- `wdt:P31` = instance of
- `wdt:P571` = inception
- `wdt:P1082` = population

> **Note** : `SERVICE wikibase:label` est un mecanisme specifique a Wikidata qui resout automatiquement les labels dans la langue demandee.

---

## 4. Requetes Federees avec SERVICE

SPARQL permet de combiner plusieurs endpoints dans une meme requete avec le mot-cle `SERVICE`.

In [ ]:
# Federated query: combine local graph with remote endpoint
from rdflib import Graph, Namespace

# Create a small local graph with French city URIs
g_local = Graph()
DBR = Namespace("http://dbpedia.org/resource/")
g_local.bind("dbr", DBR)

# Add some city URIs (we don't have data about them locally)
cities = [DBR.Paris, DBR.Lyon, DBR.Marseille]
for city in cities:
    # We'll query DBpedia for their populations
    pass

# Federated query: get population from DBpedia for our local cities
query_federated = """
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?city ?name ?population
WHERE {
    VALUES ?city { dbr:Paris dbr:Lyon dbr:Marseille }
    SERVICE <http://dbpedia.org/sparql> {
        ?city rdfs:label ?name .
        ?city dbo:populationTotal ?population .
        FILTER (lang(?name) = "fr")
    }
}
ORDER BY DESC(?population)
"""

sparql_fed = SPARQLWrapper("http://dbpedia.org/sparql")
sparql_fed.setReturnFormat(JSON)
sparql_fed.setQuery(query_federated)

try:
    results_fed = sparql_fed.query().convert()
    print("=== Villes francaises (requete federée) ===")
    for r in results_fed["results"]["bindings"]:
        name = r["name"]["value"]
        pop = int(r["population"]["value"])
        print(f"  {name} : {pop:,d} habitants")
except Exception as e:
    print(f"Erreur : {e}")

### Interpretation : Requetes federées

Le mot-cle `SERVICE` permet d'interroger un endpoint distant depuis une requete SPARQL.

```sparql
SERVICE <http://dbpedia.org/sparql> {
    # patterns a evaluer sur DBpedia
}
```

C'est puissant pour combiner des donnees locales avec des donnees distantes, ou pour joindre plusieurs endpoints entre eux.

Equivalent dotNetRDF : Meme syntaxe SPARQL, le federated query est gere par le moteur SPARQL.

---

## 5. Formats de Retour

SPARQLWrapper supporte differents formats de retour pour les requetes SPARQL.

In [ ]:
# Compare JSON vs XML return formats
query_simple = """
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>

ASK { dbr:Paris a dbo:City }
"""

# JSON format
sparql_json = SPARQLWrapper("http://dbpedia.org/sparql")
sparql_json.setReturnFormat(JSON)
sparql_json.setQuery(query_simple)

# XML format
sparql_xml = SPARQLWrapper("http://dbpedia.org/sparql")
sparql_xml.setReturnFormat(XML)
sparql_xml.setQuery(query_simple)

try:
    result_json = sparql_json.query().convert()
    print(f"Format JSON : {result_json}")
    
    result_xml = sparql_xml.query().convert()
    print(f"\nFormat XML (first 200 chars) : {result_xml[:200]}...")
except Exception as e:
    print(f"Erreur : {e}")

### Interpretation : Formats de retour

| Format | Constante SPARQLWrapper | Usage |
|--------|------------------------|-------|
| **JSON** | `JSON` | Format le plus simple a parser en Python |
| **XML** | `XML` | Format standard W3C |
| **CSV/TSV** | Necessite conversion | Pour l'export vers Excel |

Le format JSON est generalement recommande pour Python car il se convertit directement en dictionnaires.

---

## 6. Tableau de Correspondance dotNetRDF / SPARQLWrapper

Ce tableau recapitule les equivalences pour les requetes vers des endpoints distants.

| Operation | dotNetRDF (C#) | SPARQLWrapper (Python) |
|-----------|---------------|--------------------------|
| **Creer endpoint** | `new SparqlRemoteEndpoint(uri)` | `SPARQLWrapper(uri)` |
| **Format retour** | Constructeur (defaut XML) | `setReturnFormat(JSON/XML)` |
| **Definir requete** | `Query = "SELECT..."` ou `SetQuery()` | `setQuery("SELECT...")` |
| **Executer SELECT** | `QueryWithResultSet()` | `query().convert()` |
| **Executer ASK** | `AskQuery()` | `query().convert()` |
| **User-Agent** | `HttpClientHeaders` | `.agent = "..."` |
| **Timeout** | `Timeout` propriete | `setTimeout(seconds)` |

### Philosophies differentes

| Aspect | dotNetRDF | SPARQLWrapper |
|--------|-----------|----------------|
| **Resultat SELECT** | `SparqlResultSet` (type .NET) | Dictionnaire Python brut |
| **Resultat ASK** | `bool` | Dictionnaire avec cle `boolean` |
| **Type de retour** | Objets fortement types | Structures natives Python |

---

## Exercices

Mettez en pratique les SPARQLWrapper avec ces deux exercices.

### Exercice 1 : Top 10 villes francaises via Wikidata

Utilisez SPARQLWrapper pour interroger Wikidata et obtenir les 10 villes de France les plus peuplees, avec leur nom et leur population.

**Indices** :
- Classe ville : `wd:Q515` (city)
- Pays : `wdt:P17` (country), France = `wd:Q142`
- Population : `wdt:P1082`
- Pensez au `try/except` et au `User-Agent`

In [ ]:
# Exercice 1 : Top 10 villes francaises via Wikidata

sparql_ex1 = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql_ex1.setReturnFormat(JSON)
sparql_ex1.agent = "CoursIA-SemanticWeb-Notebook/1.0 (educational)"

sparql_ex1.setQuery("""
# TODO: Completez la requete SPARQL
SELECT ?cityLabel ?population
WHERE {
    # instance of: city (wd:Q515)
    # country: France (wd:Q142)
    # population (wdt:P1082)
    # SERVICE wikibase:label
}
ORDER BY DESC(?population)
LIMIT 10
""")

try:
    results_ex1 = sparql_ex1.query().convert()
    for r in results_ex1["results"]["bindings"]:
        print(f"{r['cityLabel']['value']} : {int(r['population']['value']):,d} habitants")
except Exception as e:
    print(f"Erreur : {e}")
    print("Verifiez la syntaxe de votre requete ou la disponibilite de l'endpoint.")

### Exercice 2 : Films de Christopher Nolan via DBpedia

Ecrivez une requete DBpedia pour lister les films realises par Christopher Nolan, avec leur titre et annee de sortie.

**Indices** :
- Christopher Nolan : `dbr:Christopher_Nolan`
- Propriete realisateur : `dbo:director`
- Propriete annee : `dbo:releaseDate` ou `dbo:year`

In [ ]:
# Exercice 2 : Films de Christopher Nolan

sparql_ex2 = SPARQLWrapper("http://dbpedia.org/sparql")
sparql_ex2.setReturnFormat(JSON)

sparql_ex2.setQuery("""
# TODO: Completez la requete
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>

SELECT ?film ?title ?year
WHERE {
    # Trouvez les films avec dbr:Christopher_Nolan comme realisateur
}
ORDER BY DESC(?year)
""")

try:
    results_ex2 = sparql_ex2.query().convert()
    print("=== Films de Christopher Nolan ===")
    for r in results_ex2["results"]["bindings"]:
        print(f"  {r['title']['value']} ({r['year']['value']})")
except Exception as e:
    print(f"Erreur : {e}")

---

## Resume

Ce sidetrack a presente l'interrogation de donnees liees du Web avec **SPARQLWrapper**.

### Concepts cles

| Concept | Ce que vous avez appris |
|---------|------------------------|
| **SPARQLWrapper** | Interroger des endpoints SPARQL distants |
| **DBpedia** | Donnees structurees de Wikipedia |
| **Wikidata** | Base de connaissances avec Q-items/P-properties |
| **SERVICE** | Requetes federées entre endpoints |
| **Correspondance** | Equivalences dotNetRDF / SPARQLWrapper |

### Prochaines etapes

- **SW-6-CSharp-RDFS** : Schema et inference en .NET
- **SW-7-CSharp-OWL** : Ontologies et raisonnement avance
- **SW-7b-Python-OWL** : Introduction a OWLReady2 pour Python

---

**Navigation** : [<< SW-5-CSharp-LinkedData](SW-5-CSharp-LinkedData.ipynb) | [Index](README.md) | [SW-6-CSharp-RDFS >>](SW-6-CSharp-RDFS.ipynb)